# Building an Image Classifier Deep Network applying Transfer Learning

In [1]:
# import the required libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tf.keras.utils
from PIL import Image
import numpy as np
import os
import base64
from io import BytesIO

2023-11-13 10:05:51.180700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 10:05:53.042384: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-13 10:05:53.042468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-13 10:05:53.042477: W 

## Dataset creation

In [2]:
# dataset's directory path
directory = "../dataset/images/"

batch_size = 32
image_size = (160,160) # resize the images to 160x160

# create training and validation sets
# use image_dataset_from_directory to load the images
# set a validation split and specify the subset ('training' or 'validation')
# set the random seeds to match eachother (to avoid overlapping of the images in
# train and validation sets)

train_set = tf.keras.utils.image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=image_size,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_set = tf.keras.utils.image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=image_size,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

Found 183 files belonging to 2 classes.
Using 147 files for training.
Found 183 files belonging to 2 classes.
Using 36 files for validation.


2023-11-13 10:06:05.448265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-13 10:06:05.448298: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-13 10:06:05.448321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (testtl-0): /proc/driver/nvidia/version does not exist
2023-11-13 10:06:05.450395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Obtain Class names (labels)

In [3]:
# print some images
# use .class_names attribute to retrieve the classes of the images from the dicrectories names
class_names = train_set.class_names
print(class_names)

['tea-earl-grey', 'tea-lemon']


# Save model (JVM) with Base64 Signature

### Load labels

In [14]:
print(class_names)
labels = tf.constant([class_names])
print(labels)

['tea-earl-grey', 'tea-lemon']
tf.Tensor([[b'tea-earl-grey' b'tea-lemon']], shape=(1, 2), dtype=string)
tf.Tensor(b'tea-earl-grey', shape=(), dtype=string)


### Load augmented model

In [47]:
smodel = tf.saved_model.load("../models/tea_model/1")

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
# print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    image = tf.image.resize(image, [160, 160])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = labels[0][idx[0]]
    label = tf.reshape(label,[1])
    
    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    probability = tf.reshape(tf.as_string(probability),[1])
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, probability], axis=0)
    result = tf.reshape(result,[2, 1])
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    export_dir="../models/tea_model_b64_jvm/1",
    signatures=new_signature
)

ConcreteFunction signature_wrapper(*, input_4)
  Args:
    input_4: float32 Tensor, shape=(None, 160, 160, 3)
  Returns:
    {'dense_1': <1>}
      <1>: float32 Tensor, shape=(None, 2)


INFO:tensorflow:Assets written to: ../models/tea_model_b64_jvm/1/assets


INFO:tensorflow:Assets written to: ../models/tea_model_b64_jvm/1/assets


# Test Base64 Model with single image

In [48]:
smodel = tf.saved_model.load("../models/tea_model_b64_jvm/1")

# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

ConcreteFunction signature_wrapper(*, image_b64)
  Args:
    image_b64: string Tensor, shape=(1,)
  Returns:
    {'output_0': <1>}
      <1>: string Tensor, shape=(2, 1)


### Run Inference

In [49]:
# testfile = "../dataset/images/tea-earl-grey/PXL_20230803_124233750.MP.jpg"
testfile = "../dataset/images/tea-lemon/PXL_20230803_153317298.jpg"

# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)

{'output_0': <tf.Tensor: shape=(2, 1), dtype=string, numpy=
array([[b'tea-lemon'],
       [b'0.913074']], dtype=object)>}


# Testing Tensor manipulation

In [37]:
a = tf.constant(['one', 'two'])
b = tf.constant(['three', 'four'])

print(a)
print(b)
# print(tf.reshape(a[1],[1]))
a = tf.reshape(a[1],[1])
b = tf.reshape(b[1],[1])
print(a)
print(b)
# print(tf.concat([a[1],b[1]],axis=0))

result = tf.concat([a, b], axis=0)
print(result)
# result = tf.concat([label, [tf.as_string(probability)]], axis=0)

tf.Tensor([b'one' b'two'], shape=(2,), dtype=string)
tf.Tensor([b'three' b'four'], shape=(2,), dtype=string)
tf.Tensor([b'two'], shape=(1,), dtype=string)
tf.Tensor([b'four'], shape=(1,), dtype=string)
tf.Tensor([b'two' b'four'], shape=(2,), dtype=string)
